In [2]:
import scipy.spatial.distance
import numpy as np
import torch

In [4]:
features = torch.randn(16,512)
features = np.asarray(features)

In [19]:
D = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(features)) # [16,16]

In [20]:
D = D.copy()
n = D.shape[0]

In [21]:
D += np.diag([np.nan]*n)

In [24]:
def maxD(M, allowed):
    s = np.nansum(M, axis=1)
    s[~allowed] = np.nan
    return np.nanargmax(s)

In [22]:
clust_inds = np.zeros(n, dtype=np.int) + np.nan
cur_index = 1

/home/unix/mjeon/anaconda3/envs/privacy/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


In [8]:
# takes N by N precomputed distance matrix D
# outputs a length N cluster assignment vector with indicies 1..C for C clusters
def nearestNeighborClust(D, clust_size=2):
    D = D.copy()
    n = D.shape[0]
    assert D.shape[1] == n

    D += np.diag([np.nan] * n)
    def maxD(M, allowed):
        s = np.nansum(M,axis=1)
        s[~allowed] = np.nan
        return np.nanargmax(s)
    
    clust_inds = np.zeros(n, dtype=np.int) + np.nan
    cur_index = 1
    while np.any(np.isnan(clust_inds)):
        # Choose maxD point
        next_point = maxD(D, np.isnan(clust_inds))
    
        # Find clust_size-1 closest points
        nearest = np.argsort(D[next_point,:])[:clust_size-1]
        
        # Assign cluster
        cluster = np.append(nearest, next_point).astype(np.int)
        clust_inds[cluster] = cur_index
        
        # Update matrix
        D[cluster,:] = np.nan
        D[:,cluster] = np.nan
        
        cur_index += 1
    return clust_inds.astype(np.int)

In [10]:
centroid_idx = nearestNeighborClust(predistance, clust_size=5)

/home/unix/mjeon/anaconda3/envs/privacy/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/home/unix/mjeon/anaconda3/envs/privacy/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, che

In [33]:
a = torch.randn(16,16*16)

In [36]:
a

tensor([[ 0.0360,  1.5181,  0.6306,  ...,  2.2740, -1.5352,  0.7248],
        [-0.6991, -0.1427,  0.1687,  ..., -0.9486, -1.9707, -1.5147],
        [-0.2743, -0.1444,  0.9321,  ...,  0.0250, -0.9977, -1.6982],
        ...,
        [ 0.0735,  0.4119,  0.3067,  ...,  2.1728, -0.5541, -0.4576],
        [-0.2734,  0.8776,  1.9781,  ...,  0.6326,  0.0446,  0.0163],
        [-0.5316, -1.7545, -0.6329,  ..., -1.7420,  0.6207, -1.2899]])

In [37]:
a.T

tensor([[ 0.0360, -0.6991, -0.2743,  ...,  0.0735, -0.2734, -0.5316],
        [ 1.5181, -0.1427, -0.1444,  ...,  0.4119,  0.8776, -1.7545],
        [ 0.6306,  0.1687,  0.9321,  ...,  0.3067,  1.9781, -0.6329],
        ...,
        [ 2.2740, -0.9486,  0.0250,  ...,  2.1728,  0.6326, -1.7420],
        [-1.5352, -1.9707, -0.9977,  ..., -0.5541,  0.0446,  0.6207],
        [ 0.7248, -1.5147, -1.6982,  ..., -0.4576,  0.0163, -1.2899]])

In [45]:
a = torch.tensor([[1,2],[3,4]])
print(a)
print(a.T)

tensor([[1, 2],
        [3, 4]])
tensor([[1, 3],
        [2, 4]])


In [44]:
a @ a.T

tensor([[ 5, 11],
        [11, 25]])